# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [41]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.tile_sources as gvts
import numpy as np
import geopandas as gpd

# Import API key
from api_keys import geoapify_key

In [42]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint-philippe,-21.3585,55.7679,25.77,82,52,4.34,RE,1710296680
1,1,waitangi,-43.9535,-176.5597,16.01,80,100,3.58,NZ,1710296681
2,2,nemuro,43.3236,145.5750,-3.44,74,100,18.05,JP,1710296681
3,3,kourou,5.1552,-52.6478,28.03,89,100,4.01,GF,1710296682
4,4,albany,42.6001,-73.9662,8.23,56,1,2.28,US,1710296004


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:
%%capture --no-display

map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700, frame_height = 500, size = "Humidity", scale = 0.50, color = "City")

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [44]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = {
    'Temperature': {'max': 32},  # Roughly 90F
    'Humidity': {'max': 80},  # Maximum humidity percentage
    'Cloudiness': {'max': 80}  # Maximum cloudiness percentage
}

# Drop any rows with null values
city_data_df_new = city_data_df.dropna()

# Filter the DataFrame based on the ideal conditions
ideal_cities_df = city_data_df_new[
    (city_data_df_new['Max Temp'] <= ideal_conditions['Temperature']['max']) & 
    (city_data_df_new['Humidity'] <= ideal_conditions['Humidity']['max']) & 
    (city_data_df_new['Cloudiness'] <= ideal_conditions['Cloudiness']['max'])
]

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,albany,42.6001,-73.9662,8.23,56,1,2.28,US,1710296004
6,6,port-aux-francais,-49.3500,70.2167,5.23,68,23,15.19,TF,1710296684
7,7,invercargill,-46.4000,168.3500,12.79,75,72,4.47,NZ,1710296684
9,9,new norfolk,-42.7826,147.0587,20.40,35,1,0.61,AU,1710296686
10,10,grytviken,-54.2811,-36.5092,3.95,77,24,3.75,GS,1710296687
...,...,...,...,...,...,...,...,...,...,...
566,566,cooper's town,26.8667,-77.5167,21.10,64,65,3.33,BS,1710297068
577,577,umluj,25.0213,37.2685,24.39,51,24,2.50,SA,1710297075
578,578,winnemucca,40.9730,-117.7357,8.03,49,75,5.14,US,1710297076
579,579,pasuquin,18.3328,120.6177,29.97,51,40,3.09,PH,1710297077


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.loc[:, ['City', 'Country', 'Lat', 'Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,albany,US,42.6001,-73.9662,56,
6,port-aux-francais,TF,-49.3500,70.2167,68,
7,invercargill,NZ,-46.4000,168.3500,75,
9,new norfolk,AU,-42.7826,147.0587,35,
10,grytviken,GS,-54.2811,-36.5092,77,
...,...,...,...,...,...,...
566,cooper's town,BS,26.8667,-77.5167,64,
577,umluj,SA,25.0213,37.2685,51,
578,winnemucca,US,40.9730,-117.7357,49,
579,pasuquin,PH,18.3328,120.6177,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel

radius = 10000
categories = "accommodation.hotel"
limit = 1

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
invercargill - nearest hotel: The Grand
new norfolk - nearest hotel: Woodbridge on the Derwent
grytviken - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
zhanaozen - nearest hotel: No hotel found
mhamid - nearest hotel: Hotel Kasbah Azalay
date - nearest hotel: ホテル ローヤル
bethel - nearest hotel: Hampton Inn Danbury
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
yelizovo - nearest hotel: Николина Усадьба
blackmans bay - nearest hotel: Villa Howden
badger - nearest hotel: Clarion Hotel & Suites
manghit - nearest hotel: Mang'it miymanxanasi
papatowai - nearest hotel: No hotel found
narimanov - nearest hotel: No hotel found
eusebio ayala - nearest hotel: San Francisco Country Club
hamilton - nearest hotel: North Vista Manor
lufilufi - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
adam - nearest hotel: No hotel found
us

,City,Country,Lat,Lng,Humidity,Hotel Name
4,albany,US,42.6001,-73.9662,56,No hotel found
6,port-aux-francais,TF,-49.3500,70.2167,68,Keravel
7,invercargill,NZ,-46.4000,168.3500,75,The Grand
9,new norfolk,AU,-42.7826,147.0587,35,Woodbridge on the Derwent
10,grytviken,GS,-54.2811,-36.5092,77,No hotel found
...,...,...,...,...,...,...
566,cooper's town,BS,26.8667,-77.5167,64,No hotel found
577,umluj,SA,25.0213,37.2685,51,No hotel found
578,winnemucca,US,40.9730,-117.7357,49,The Overland Hotel
579,pasuquin,PH,18.3328,120.6177,51,Villa del Marj


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

map_plot_2 = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700, frame_height = 500, hover_cols=['Hotel Name', 'Country'], size = "Humidity", scale = 0.50, color = "City")

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)